# Import necessary lib. 


In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.contrib.layers import l1_l2_regularizer

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

import os
import h5py
import time
# from batchnorm import batch_norm,fc_batch_norm

from sklearn.metrics import accuracy_score,classification_report
import Model_1229 as my_model

E:\Liulicong\Software\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


# Configuration
Set the class names and num (17).
Set the data path (relative path). 

In [2]:
# configuration
# data attribution
CLASS_NUM = 17
CLASS_URBAN = ['Urban_Compact high-rise', 'Urban_Compact mid-rise', 'Urban_Compact low-rise',
               'Urban_Open high-rise', 'Urban_Open mid-rise', 'Urban_Open low-rise',
               'Urban_Lightweight low-rise', 'Large low-rise', 'Sparesly built', 'Heavy industry']
CLASS_RURAL = ['Dense trees', 'Scattered trees', 'Bush and scrub', 'Low plants', 'Bare rock or paved', 'Bare soil or sand', 'Water']
CLASS_NAME = CLASS_URBAN + CLASS_RURAL

# data path
base_dir = r'../data/'
path_training = os.path.join(base_dir, 'training.h5')
path_validation = os.path.join(base_dir, 'validation.h5')
path_test = os.path.join(base_dir, 'round1_test_a_20181109.h5')
# random
np.random.seed(0)

# Read Data
Check the data foramt.

In [3]:
h5py_training = h5py.File(path_training,'r')
h5py_validation = h5py.File(path_validation,'r')
h5py_test = h5py.File(path_test, 'r')

### get s1 image channel data
### it is not really loaded into memory. only the indexes have been loaded.
print("-" * 60)
print("training part")
s1_training = h5py_training['sen1']
print('s1_training: ', s1_training.shape)
s2_training = h5py_training['sen2']
print('s2_training: ', s2_training.shape)
label_training = h5py_training['label']
print('label_training: ', label_training.shape)

print("-" * 60)
print("validation part")
s1_validation = h5py_validation['sen1']
print('s1_validation: ', s1_validation.shape)
s2_validation = h5py_validation['sen2']
print('s2_validation: ', s2_validation.shape)
label_validation = h5py_validation['label']
print('label_validation: ', label_validation.shape)

print("-" * 60)
print("Test part")
s1_test = h5py_test['sen1']
print('s1_test: ', s1_test.shape)
s2_test = h5py_test['sen2']
print('s2_test: ', s2_test.shape)


train_num, pixel_x, pixel_y, sar_bands = s1_training.shape
vali_num, pixel_x, pixel_y, optical_bands = s2_validation.shape
test_num = s1_test.shape[0]

------------------------------------------------------------
training part
s1_training:  (352366, 32, 32, 8)
s2_training:  (352366, 32, 32, 10)
label_training:  (352366, 17)
------------------------------------------------------------
validation part
s1_validation:  (24119, 32, 32, 8)
s2_validation:  (24119, 32, 32, 10)
label_validation:  (24119, 17)
------------------------------------------------------------
Test part
s1_test:  (4838, 32, 32, 8)
s2_test:  (4838, 32, 32, 10)


# Pre Training

In [4]:
PRE_MODEL_prefix = "0105_PRETRAINING_NOGRAFT"
PRE_MODEL_name = "{}".format(PRE_MODEL_prefix)
my_model.build_model(PRE_MODEL_name)
my_model.training((s2_training, label_training), (s2_validation, label_validation), 
                  model_name=PRE_MODEL_name, epochs=10)

Model Build Start!
Model Build Success!
epochs : 0 Valid With AC: 0.5979210374769101 With Loss: 1.367008444098346
epochs : 1 Valid With AC: 0.6100795204974403 With Loss: 1.437443814708046
epochs : 2 Valid With AC: 0.5879043700281253 With Loss: 1.5444386043063818
epochs : 3 Valid With AC: 0.5911705661411127 With Loss: 1.5774856068225112
epochs : 4 Valid With AC: 0.5980002089472841 With Loss: 1.6852809859031894
epochs : 5 Valid With AC: 0.6040745919400995 With Loss: 1.7616489830635502
epochs : 6 Valid With AC: 0.6055042024486321 With Loss: 1.7778693282627447
epochs : 7 Valid With AC: 0.5964695446747393 With Loss: 1.876166952254884
epochs : 8 Valid With AC: 0.5812348105198095 With Loss: 2.0190632425239663
epochs : 9 Valid With AC: 0.5695652493760606 With Loss: 1.9167147102262643
Training time using: 1133.6588416099548


# One Model Training

In [49]:
ONE_MODEL_prefix = "1229_ONEMODEL"
ONE_MODEL_name = "{}".format(PRE_MODEL_prefix)
my_model.build_model(PRE_MODEL_name)
my_model.training((s2_validation[5000:, :, :, :], label_validation[5000:,:]), 
                  (s2_validation[:5000, :, :, :], label_validation[:5000,:]), 
                  model_name=ONE_MODEL_name, epochs=50, early_stop=5, is_shuffle=True)

Model Build Start!
Model Build Success!
epochs : 0 Valid With AC: 0.11339999973773957 With Loss: 5.050211242675781
epochs : 1 Valid With AC: 0.11339999973773957 With Loss: 5.938496801757813
epochs : 2 Valid With AC: 0.12200000077486038 With Loss: 5.096493896484375
epochs : 3 Valid With AC: 0.18259999960660933 With Loss: 4.234229638671875
epochs : 4 Valid With AC: 0.49139999866485595 With Loss: 1.911457586669922
epochs : 5 Valid With AC: 0.7699999976158142 With Loss: 0.7632871765136718
epochs : 6 Valid With AC: 0.8138000035285949 With Loss: 0.6028267807006836
epochs : 7 Valid With AC: 0.8446000003814698 With Loss: 0.47860005493164065
epochs : 8 Valid With AC: 0.8738000011444091 With Loss: 0.38220713348388674
epochs : 9 Valid With AC: 0.8416000056266785 With Loss: 0.46222345123291014
epochs : 10 Valid With AC: 0.8867999958992004 With Loss: 0.35815686492919924
epochs : 11 Valid With AC: 0.884399995803833 With Loss: 0.35007618560791015
epochs : 12 Valid With AC: 0.8778000020980835 With Los

# CV Training

In [4]:
MODEL_Prefix = "0105_PRET_5CV_ES5"
N = 5
kflod_turns = 1

In [5]:
from sklearn.model_selection import StratifiedKFold
for kflod_turn in range(kflod_turns):
    sfolder = StratifiedKFold(n_splits=N, random_state=kflod_turn, shuffle=False)
    for i, (train_loc, vali_loc) in enumerate(sfolder.split(np.zeros((vali_num, 1)), np.zeros(vali_num))):
        _id = i + N * kflod_turn
        print(_id, '; Train: %s | test: %s' % (train_loc, vali_loc))
        
        X_optical_train = s2_validation[train_loc, :, :, :]
        y_train = label_validation[train_loc, :]
        
        X_optical_vali = s2_validation[vali_loc, :, :, :]
        y_vali = label_validation[vali_loc, :]
        
        my_model.build_model("{}{}".format(MODEL_Prefix, _id))
        my_model.training((X_optical_train, y_train), (X_optical_vali, y_vali), is_freeze=False, 
                          model_name="{}{}".format(MODEL_Prefix, _id), epochs=100, early_stop=5, 
                          is_shuffle=True, restore_name="0105_PRETRAINING_NOGRAFT")

0 ; Train: [ 4824  4825  4826 ... 24116 24117 24118] | test: [   0    1    2 ... 4821 4822 4823]
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ..\ckpt\0105_PRETRAINING_NOGRAFT.ckpt
epochs : 0 Valid With AC: 0.858666672706604 With Loss: 0.4546046628683163
epochs : 1 Valid With AC: 0.8966666626930236 With Loss: 0.3013113264618426
epochs : 2 Valid With AC: 0.9092000007629395 With Loss: 0.2529702864673798
epochs : 3 Valid With AC: 0.9091999983787536 With Loss: 0.25894343240146417
epochs : 4 Valid With AC: 0.9230666685104371 With Loss: 0.22519907478867082
epochs : 5 Valid With AC: 0.9303333282470703 With Loss: 0.19921552383682226
epochs : 6 Valid With AC: 0.9418666648864746 With Loss: 0.169266757680409
epochs : 7 Valid With AC: 0.9335999989509582 With Loss: 0.1957737284513255
epochs : 8 Valid With AC: 0.9373999977111817 With Loss: 0.17196482696738805
epochs : 9 Valid With AC: 0.9451333308219909 With Loss: 0.1514920002192407
epochs : 10 Valid With AC: 0.94

epochs : 6 Valid With AC: 0.9413333296775818 With Loss: 0.17679359505623332
epochs : 7 Valid With AC: 0.9344666671752929 With Loss: 0.19422792469091082
epochs : 8 Valid With AC: 0.9416666650772094 With Loss: 0.1668494080825034
epochs : 9 Valid With AC: 0.9458666634559632 With Loss: 0.1636868167674759
epochs : 10 Valid With AC: 0.9472666621208191 With Loss: 0.1475721575925204
epochs : 11 Valid With AC: 0.9461333274841308 With Loss: 0.17295047388741033
epochs : 12 Valid With AC: 0.9480666589736938 With Loss: 0.15102794969062108
epochs : 13 Valid With AC: 0.9418666696548462 With Loss: 0.16218546324503758
epochs : 14 Valid With AC: 0.9363333320617676 With Loss: 0.21122746690984193
epochs : 15 Valid With AC: 0.9539999985694885 With Loss: 0.12814100306623216
epochs : 16 Valid With AC: 0.9530666661262512 With Loss: 0.1454553198853931
epochs : 17 Valid With AC: 0.9442666649818421 With Loss: 0.18323351741825566
epochs : 18 Valid With AC: 0.9524666643142701 With Loss: 0.13697873429081728
epochs 

# CV Test 

In [6]:
MODEL_Prefix = "0105_PRET_5CV_ES5"
N = 5

In [7]:
CV_proba = np.zeros((test_num, CLASS_NUM))
for i in range(N * kflod_turns):
    my_model.build_model("{}{}".format(MODEL_Prefix, i))
    CV_proba += my_model.batch_predict(s2_test, "{}{}".format(MODEL_Prefix, i))

Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/0105_PRET_5CV_ES50.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/0105_PRET_5CV_ES51.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/0105_PRET_5CV_ES52.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/0105_PRET_5CV_ES53.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/0105_PRET_5CV_ES54.ckpt


In [8]:
CV_proba_max_loc = np.argmax(CV_proba, 1)
one_hot=tf.one_hot(CV_proba_max_loc,CLASS_NUM)
with tf.Session() as sess:
    _one_hot = sess.run(one_hot)
_one_hot = _one_hot.astype(np.int16)
pd.DataFrame(_one_hot).to_csv('../result/0105_PreT_5CV.csv'.format(N), index=False, header=None)